In [ ]:
import json
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [ ]:
def filter_df(df, num_sources=None, num_targets=None):
    """
    A function to filter the data frame by top n sources and targets
    If num_sources or num_targets args are not supplied, they will not be filtered
    """
    if num_targets:
        top_targets = df.sum().sort_values(ascending=False)
        df = df[top_targets[:num_targets].index]
    
    if num_sources:
        top_sources = df.sum(axis=1).sort_values(ascending=False)[:num_sources]
        df = df.loc[top_sources.index]
    
    return df


def create_sankey_df(df, min_val=0):
    """
    Create the human-readable form of the Sankey chart data from an input data frame
    Data can be filtered by a threshold minimum value
    | Source | Source Value | Target | Target Value |
    |    A   |      5       |   i    |      3       |
    |    A   |      5       |   j    |      2       |
    |    B   |      7       |   i    |      1       |
    |    B   |      7       |   k    |      4       |
    """
    
    sources = []
    source_vals = []
    targets = []
    target_vals = []
    for source_name in df.index:
        row = df.loc[source_name]
        sources += [source_name] * sum(row.values > min_val)
        source_vals += [row[row.values > min_val].sum()] * sum(row.values > min_val)
        targets += list(row[row > min_val].index)
        target_vals += list(row[row > min_val].values)
    
    sankey_df = pd.DataFrame({
        'source': sources,
        'target': targets,
        'value': target_vals
    })
    
    return sankey_df


def create_label_dict(node_df, start_idx=0):
    """
    Return a dictionary with labels as keys and indices as values.  Applied 
    to each section of the flow visualization (two nodes and an edge). The
    `node_df` represents each section.
    """
    labels = set(node_df.source).union(node_df.target)
    sorted_labels = sorted(list(labels))
    return {sorted_labels[i]: i + start_idx for i in range(len(sorted_labels))}


def create_final_list(node_df, node_label_dict):
    """
    Return a list of dictionaries and the labels for each section of the flow 
    diagram.
    """
    df = pd.DataFrame({
        "source": node_df.source.map(node_label_dict),
        "target": node_df.target.map(node_label_dict),
        "value" : node_df.value
    })
    
    labels = list(node_label_dict.keys())
    return df.to_dict("records"), labels

In [ ]:
investor = pd.read_excel(
    "../data/Equity investor SUP matrix.xlsx",
    engine="openpyxl",
    skiprows=3,
    usecols="B, E:GG",
)

investor = investor.rename(columns={investor.columns[0]: "Ultimate Investor"})

# drop last row because it is a table summary
investor = investor[:-1]
investor = investor.set_index('Ultimate Investor')

# Set the value of Kingdom of Saudi Arabia investment in Saudi Arabian Oil company to equal
# the second largest investment
investor.loc['KINGDOM OF SAUDI ARABIA', ('Saudi Arabian Oil Co')] = investor.max(axis=0).sort_values(ascending=False)[1]

In [ ]:
financer = pd.read_excel(
    "../data/Financing SUP matrix.xlsx",
    engine="openpyxl",
    skiprows=4,
    usecols="A:AV",
)

# drop last row because it is null
financer = financer[:-1]
financer = financer.set_index('Bank')

In [ ]:
producer = pd.read_excel(
    "../data/MFA matrix.xlsx",
    sheet_name="Conversion",
    engine="openpyxl",
    skiprows=1,
    usecols="C:FY",
).dropna()

producer = producer.groupby('Producer').sum()

In [ ]:
waste = pd.read_excel(
    "../data/MFA matrix.xlsx",
    sheet_name="Waste",
    engine="openpyxl",
    skiprows=1,
    usecols="B, D:FY",
).dropna()

waste = waste.groupby('Country').sum()

In [ ]:
destination = pd.read_excel(
    "../data/MFA matrix.xlsx",
    sheet_name="Waste",
    engine="openpyxl",
    skiprows=1,
    usecols="C:FY",
).dropna()

destination = destination.groupby('Producer').sum()

In [ ]:
investor_df = create_sankey_df(investor)
financier_df = create_sankey_df(financer)
producer_df = create_sankey_df(producer)
waste_df = create_sankey_df(waste)
destination_df = create_sankey_df(destination)

# *Important!* Choose to export either `investor` or `financier` dataset here:

In [ ]:
#dataset = 'investor'
dataset = 'financier'

# Export JSON for D3

In [ ]:
# Concatenate two (or more) dataframes together

if dataset == 'investor':
    df_1 = investor_df
if dataset == 'financier':
    df_1 = financier_df
else:
    print("Choose either 'investor' or 'financier' as the dataset type")

df_2 = destination_df

full_df = pd.concat([df_1, df_2], ignore_index=True)

column_dict = {name: 1 for name in np.unique(df_1['source']) }
for name in np.unique(pd.concat([df_1['target'], df_2['source']])):
    column_dict[name] = 2
for name in np.unique(df_2['target']):
    column_dict[name] = 3
    
# For every unique name in dataframe, create a name to index dictionary
labels = np.unique([[full_df['source']] + [full_df['target']]])
total_idx_dict = { labels[i]: i for i in range(len(labels)) }

In [ ]:
# Generate base structure for dataset and populate links
full_d3_dataset = {
    'links': [],
    'nodes': []
}

for src, tar, val in zip(list(full_df.source.map(total_idx_dict)), 
                         list(full_df.target.map(total_idx_dict)), 
                         list(full_df.value)):
    full_d3_dataset['links'].append({
        "source": src,
        "target": tar,
        "value": round(val, 3)
    })

for label in labels:
    full_d3_dataset['nodes'].append({
        "name": label,
        "column": column_dict[label],
        "hover": {},
        "locations": []
    })

## Add Hover Label Info

In [ ]:
# Add investor value data
investor_value = pd.read_excel(
    "../data/Equity investor SUP matrix.xlsx",
    engine="openpyxl",
    skiprows=3,
    usecols="B, GH",
)

investor_value = investor_value.rename(columns={investor_value.columns[0]: "Ultimate Investor"})
investor_value = investor_value[:-1]
investor_value = investor_value.set_index('Ultimate Investor')

for name in investor_value.index:
    node_names = [node['name'] for node in full_d3_dataset['nodes']]
    if name in node_names:
        d3_node_index = node_names.index(name)
        value = investor_value.loc[name]['Holding - Single Use Plastic Adjusted (USD mn)']
        full_d3_dataset['nodes'][d3_node_index]['hover']['Total Value'] = round(value, 3)

        
# Add investor volume data
investor_volume = pd.read_excel(
    "../data/Equity investor SUP footprint matrix.xlsx",
    engine="openpyxl",
    skiprows=3,
    usecols="B, CV",
)

investor_volume = investor_volume.rename(columns={investor_volume.columns[0]: "Ultimate Investor"})
investor_volume = investor_volume[:-1]
investor_volume = investor_volume.set_index('Ultimate Investor')

for name in investor_volume.index:
    node_names = [node['name'] for node in full_d3_dataset['nodes']]
    if name in node_names:
        d3_node_index = node_names.index(name)
        value = investor_volume.loc[name]['Footprint (kt)']
        full_d3_dataset['nodes'][d3_node_index]['hover']['Total Volume'] = round(value, 3)

        
# Add financer value data
financer_value = pd.read_excel(
    "../data/Financing SUP matrix.xlsx",
    engine="openpyxl",
    skiprows=4,
    usecols="A:AV",
)

financer_value = financer[:-1]
financer_value = financer_value.sum(axis=1)
for name in financer_value.index:
    node_names = [node['name'] for node in full_d3_dataset['nodes']]
    if name in node_names:
        d3_node_index = node_names.index(name)
        value = financer_value[name]
        full_d3_dataset['nodes'][d3_node_index]['hover']['Total Value'] = round(value, 3)        


# Add producer volume label
producer_total_volume = pd.read_excel(
    "../data/MFA matrix.xlsx",
    sheet_name="Pivot - Waste",
    engine="openpyxl",
    skiprows=2,
    usecols="A:B",
).dropna().set_index('Row Labels')

for name in producer_total_volume.index:
    node_names = [node['name'] for node in full_d3_dataset['nodes']]
    if name in node_names:
        d3_node_index = node_names.index(name)
        value = producer_total_volume.loc[name]['Sum of checksum']
        full_d3_dataset['nodes'][d3_node_index]['hover']['Total Volume'] = round(value, 3) 


# Add producer equity value label
equity_value = pd.read_excel(
    "../data/Equity investor SUP matrix.xlsx",
    engine="openpyxl",
    skiprows=3,
    usecols="B, E:GG",
)

equity_value = equity_value.rename(columns={equity_value.columns[0]: "Ultimate Investor"})
equity_value = equity_value[:-1]
equity_value = equity_value.set_index('Ultimate Investor')
equity_value = equity_value.sum()
equity_value.head()

for name in equity_value.index:
    node_names = [node['name'] for node in full_d3_dataset['nodes']]
    if name in node_names:
        d3_node_index = node_names.index(name)
        value = equity_value[name]
        full_d3_dataset['nodes'][d3_node_index]['hover']['Equity Value'] = round(value, 3)
        
        
# Add circularity score label
circularity = pd.read_excel(
    "../data/CA matrix.xlsx",
    sheet_name='Circularity Assessment',
    engine="openpyxl",
    skiprows=1,
    usecols="D, F",
).dropna().set_index('Polymer producers ranked by circularity score')

for name in circularity.index:
    node_names = [node['name'] for node in full_d3_dataset['nodes']]
    if name in node_names:
        d3_node_index = node_names.index(name)
        value = circularity.loc[name]['Circularity\nscore']
        full_d3_dataset['nodes'][d3_node_index]['hover']['Circularity Score'] = value


# Add total financing value label
financer = pd.read_excel(
    "../data/Financing SUP matrix.xlsx",
    engine="openpyxl",
    skiprows=4,
    usecols="A:AV",
)
financer = financer[:-1]
financer = financer.set_index('Bank')
financing_total = financer.sum()

for name in financing_total.index:
    node_names = [node['name'] for node in full_d3_dataset['nodes']]
    if name in node_names:
        d3_node_index = node_names.index(name)
        value = financing_total[name]
        full_d3_dataset['nodes'][d3_node_index]['hover']['Total Financing'] = round(value, 3)
        
        
# Add total waste label
total_waste = pd.read_excel(
    "../data/MFA matrix.xlsx",
    sheet_name="Waste",
    engine="openpyxl",
    skiprows=1,
    usecols="B:C, D:FY",
).dropna()
total_waste = total_waste.groupby('Producer').sum()
total_waste = total_waste.sum()

for name in total_waste.index:
    node_names = [node['name'] for node in full_d3_dataset['nodes']]
    if name in node_names:
        d3_node_index = node_names.index(name)
        value = total_waste[name]
        full_d3_dataset['nodes'][d3_node_index]['hover']['Total Waste'] = round(value, 3)

## Assign Regions to Countries

In [ ]:
with open('../data/region_map.json', 'r') as f:
    region_dict = json.load(f)
country_to_region = { country['name']: country['region'] for country in region_dict }

for node in full_d3_dataset['nodes']:
    if node['name'] in country_to_region.keys():
        node['region'] = country_to_region[node['name']]

In [ ]:
print("Nodes without a matched region:", [node['name'] for node in full_d3_dataset['nodes'] if 'region' not in list(node.keys()) and node['column'] == 3])

## Add Producer Coordinates

In [ ]:
locations = pd.read_csv('../data/producer_locations.csv')

node_names = [node['name'] for node in full_d3_dataset['nodes']]
for index in range(len(locations)):
    row = locations.loc[index]
    producer_name = row['Producer name']
    lon = row['city_lon']
    lat = row['city_lat']
    coords = []
    if producer_name in node_names:
        d3_node_index = node_names.index(producer_name)
        full_d3_dataset['nodes'][d3_node_index]['locations'].append([lat, lon])

## Inspect Node Data and Export

In [ ]:
full_d3_dataset['nodes'][:15]

In [ ]:
if dataset == 'investor':
    with open('../data/d3/investor-producer-destination_labels_coords_regions.json', 'w') as f:
        json.dump(full_d3_dataset, f)
if dataset == 'financier':
    with open('../data/d3/financier-producer-destination_labels_coords_regions.json', 'w') as f:
        json.dump(full_d3_dataset, f)
